In [ ]:
import cobrame
import cobra
import pickle
import math
import json
from glob import glob
from tqdm import tqdm
from os.path import exists
import seaborn as sns
from matplotlib import pyplot as plt
import palettable
import numpy as np
from collections import OrderedDict
from scipy.stats import ttest_ind, ks_2samp, ranksums, mannwhitneyu, spearmanr, pearsonr
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering

from me_biomass.general_info import met_to_name, aas_to_name
from me_biomass.me_solve_functions import solve_me_model
from me_biomass.load_model import load_me_model
from me_biomass.characterize_solution import *
from me_biomass.load_model import currency_met_to_synthesis_rxn
from me_biomass.update_mpl_rcparams import update_rcparams

from cobrame.util.dogma import amino_acids

%matplotlib inline

def to_drop(x):
    if x.startswith('charging') or x.startswith('formation') or x.startswith('translation'):
        return True
    elif x.startswith('transcription') or x.startswith('DM_'):
        return True
    else:
        return False

In [ ]:
out_loc = './output/'

# Part 1
Run the following to reproduce figures in
- Section 1: Benchmarking ME-model predictions of biomass composition
- Section 2: Growth condition-dependent biomass composition

In [ ]:
source_to_name = {'C': 'Carbon', 'P': 'Phosphorus', 'S': 'Sulfur', 'N': 'Nitrogen'}
# filter metal ions and metabolites whose primary role is not as a prosthetic group
prosthetic_group_mets_to_filter = [
    'lys__L_c','nad_c', 'nadp_c', 'ca2_c', 'cl_c', 'cobalt2_c', 'cu2_c', 'fe2_c',
    'fe3_c', 'k_c', 'mg2_c', 'nadh_c', 'nadph_c', 'ni2_c', 'zn2_c', 'mn2_c', 'fad_c', 'nh4_c']
aas = [i[:-2] for i in amino_acids.values()]

In [ ]:
me = load_me_model(json=False)

# (1) Benchmarking ME-model predictions of biomass composition

In [ ]:
# run aerobic and anaerobic ME-model simulations
if not exists('./simulations/aerobic.json'):
    me.reactions.EX_o2_e.lower_bound = -1000
    solve_me_model(me, 2.5, using_soplex=False)
    sol1 = me.solution
    me.reactions.EX_o2_e.lower_bound = 0
    solve_me_model(me, 2.5, using_soplex=False)
    sol2 = me.solution
    with open('./simulations/aerobic.json', 'w') as f:
        json.dump(sol1.x_dict, f)
    with open('./simulations/anaerobic.json', 'w') as f:
        json.dump(sol2.x_dict, f)
else:
    with open('./simulations/aerobic.json', 'r') as f:
        x_dict = json.load(f)
    sol1 = cobra.core.Solution(1, x_dict=x_dict, status='optimal')
    with open('./simulations/anaerobic.json', 'r') as f:
        x_dict = json.load(f)
    sol2 = cobra.core.Solution(1, x_dict=x_dict, status='optimal')

In [ ]:
# plot comparision of ME-model simulations to iJO1366 biomass function
fig, axes = plt.subplots(3, 1, figsize=(10, 11))
update_rcparams()
met_list = []
for i, kind in enumerate(['amino_acid', 'cofactors', 'coenzymes']):
    if kind in ['amino_acid', 'cofactors']:
        comp1 = compare_to_ijo_biomass(me, solution=sol1, kind=kind)
        comp2 = compare_to_ijo_biomass(me, solution=sol2, kind=kind)
        if kind == 'cofactors':
            for met in prosthetic_group_mets_to_filter: 
                comp1.drop([met], inplace=True, errors='ignore')
                comp2.drop([met], inplace=True, errors='ignore')
    else:
        comp1 = compare_cofactor_to_ijo_biomass(me, currency_met_to_synthesis_rxn=currency_met_to_synthesis_rxn,
                                                solution=sol1)
        comp2 = compare_cofactor_to_ijo_biomass(me, currency_met_to_synthesis_rxn=currency_met_to_synthesis_rxn, 
                                                solution=sol2)
        
    logy = True
    joined = comp1.join(comp2, rsuffix='_ana')
    joined = joined.drop('gthox', errors='ignore')

    joined.rename(lambda x: x.replace("_c", ''), inplace=True)
    joined.drop('Measured_ana', axis=1, inplace=True)

    joined.columns = ['ME-Aerobic', 'iJO1366 BOF', 'ME-Anaerboic']
    
    ax = axes[i]
    
    legend=False
    if kind == 'amino_acid':
        title = 'Amino Acids'
        legend = True
    elif kind == 'cofactors':
        title = 'Prosthetic Groups'
    elif kind == 'coenzymes':
        title = 'Coenzymes'
    joined.rename(met_to_name, inplace=True)
    plt.rcParams['legend.facecolor'] = 'w'
    joined[['ME-Aerobic', 'ME-Anaerboic', 'iJO1366 BOF']].plot(kind='bar', logy=logy, ax=ax,
                                                               color=['#0099E6', '#F23814', '#000000'],
                                                               legend=legend)
    
    ax.set_xticklabels(joined.index, fontdict={'horizontalalignment': 'right', 'rotation':45})
    ax.set_title(title)
    ax.set_facecolor('w')
    met_list.extend([i for i in joined.index])
axes[0].set_ylim([.01, 10])
axes[1].set_ylabel(r'Growth Normalized Synthesis ($\mathrm{\frac{mmol}{gDW}}$)', size=20)
ax.figure.tight_layout()
ax.figure.subplots_adjust(hspace=1)
ax.figure.savefig('%s/figure_2.png' % out_loc)
ax.figure.savefig('%s/Fig2.svg' % out_loc)

# (2) Growth condition-dependent biomass composition

In [ ]:
def rename_columns(df):
    full = False
    col_names = []
    for i in df.columns:
        if 'iron' in i:
            col_names.append('iron')
            continue
        try:
            met = me.metabolites.get_by_id(i + '_c')
        except:
            met = me.metabolites.get_by_id(i)
        if not met.name:
            met.name = met.id.replace("_c", '')
        elif met.id == 'nadp_c':
            met.name = 'nadp'
        elif met.id == 'nad_c':
            met.name = 'nad'
        elif met.id == 'fad_c':
            met.name = 'fad'
        if not full:
            met.name = met.id.replace("_c", "")
        col_names.append(met.name)

    df.columns = col_names
    return df


def get_cofactor_demand(df, growth_norm):
    
    out_dict = {}
    for c in tqdm(df.columns):
        x_dict = df[c].to_dict()
        sol = cobra.core.Solution(1, x_dict=x_dict, status='optimal')
        out_dict[c] = {}
        
        # get coenzyme, prosthetic group, and amino acid demands
        prosthetic = compare_to_ijo_biomass(me, solution=sol, kind='cofactors', 
                                           growth_norm=growth_norm)
        coenzymes = compare_cofactor_to_ijo_biomass(
            me, currency_met_to_synthesis_rxn=currency_met_to_synthesis_rxn, 
            solution=sol, growth_norm=growth_norm)
        
        aas = compare_to_ijo_biomass(me, solution=sol, kind='amino_acid', growth_norm=growth_norm)
        
        # update output with biomass component demands
        out_dict[c].update(aas.drop('Measured', axis=1).to_dict().popitem()[1])
        out_dict[c].update(coenzymes.drop('Measured', axis=1).drop(
            prosthetic_group_mets_to_filter, errors='ignore').to_dict().popitem()[1])
        out_dict[c].update(prosthetic.drop('Measured', axis=1).drop(
            prosthetic_group_mets_to_filter, errors='ignore').to_dict().popitem()[1])
    return pd.DataFrame(out_dict).T


def get_color_for_conditions(cluster_df):
    color_df = pd.DataFrame()
    color_dict = {'N': 'g', 'C': 'k', 'P':'r', 'S':'y'}
    ana_color_dict = {'N': '#90ee90', 'C': '#D3D3D3', 'P': '#ff6961', 'S': '#ffffe0'}
    for i in cluster_df.index:
        if 'anaerobic' in i:
            color_df.loc[i, 'Nutrient Source'] = ana_color_dict[i.split('_')[0]]
        else:
            color_df.loc[i, 'Nutrient Source'] = color_dict[i.split('_')[0]]
    return color_df


def summarize_conditions(df, growth_norm=True, split=False):

    condition_df = get_cofactor_demand(df, growth_norm)
        
    # these do not consider glutathione as a substrate since it is a cofactor
    #condition_df.drop(['C_gthrd_e', 'S_gthrd_e', 'N_gthrd_e', 
    #                 'C_anaerobic_gthrd_e', 'S_anaerobic_gthrd_e', 
    #                 'N_anaerobic_gthrd_e'], inplace=True, errors='ignore')
    

    condition_df = rename_columns(condition_df)
    return condition_df


In [ ]:
full_dict = {}
for fi in glob('./simulations/media_sims/*'):
    with open(fi, 'r') as f:
        x_dict = json.load(f)
    fi = fi.split('/')[-1].replace('_sol.json', '')
    if 'anaerobic' in fi:
        name = fi.replace('anaerobic_', '').replace('_EX_', '_anaerobic_')
    else:
        name = fi.replace('aerobic_', '').replace('_EX_', '_')
    full_dict[name] = x_dict
    
full_df = pd.DataFrame(full_dict)
df = full_df.dropna(axis=1).copy()
df.to_csv('%s/raw_solutions.csv' % out_loc)

In [ ]:
full_df = full_df.fillna(0)
reorg_df = pd.DataFrame()
for i in full_df.columns:
    if 'anaerobic' in i:
        met = i.split('aerobic_')[-1][:-2]
        source = source_to_name[i[0]] + ' anaerobic'
    else:
        met = '_'.join(i.split('_')[1:-1])
        source = source_to_name[i[0]] + ' aerobic'
    reorg_df.loc[source, met] = full_df.loc['biomass_dilution', i]
    
reorg_df.T.fillna('').to_excel('%s/S1_data.xlsx' % out_loc)

In [ ]:
print('total number of conditions', reorg_df.clip(lower=1, upper=1).sum().sum())
print('total number of aerobic conditions', reorg_df.loc[[i for i in reorg_df.index if 'anaerobic' not in i]].clip(lower=1, upper=1).sum().sum())
growth_support_df = reorg_df.copy()
growth_support_df[growth_support_df>0.01] =1
print('total number of growth-supporting aerobic conditions', growth_support_df.loc[[i for i in reorg_df.index if 'anaerobic' not in i]].sum().sum())
print('total number of growth-supporting conditions', growth_support_df.sum().sum())

In [ ]:
# plot distribution of growth rates by nutrient conditions
fig, axes = plt.subplots(3, 3, figsize=(15, 10), sharex=True)
iter_axes = iter(axes.flatten())
hist_df = reorg_df.copy()
hist_df[hist_df == 0] = np.nan
for index in hist_df.sort_index().index:
    if 'Carbon' not in index and 'Nitrogen' not in index:
        bins = 2
    else:
        bins = 10
    ax = next(iter_axes)
    _ = hist_df.loc[index].hist(ax=ax, bins=bins)
    ax.set_title(index)
axes.flatten()[-1].remove()
fig.savefig('%s/growth_rates.png' % out_loc)

In [ ]:
plt.scatter(df.loc['biomass_dilution'], df.loc['protein_biomass_to_biomass'])
plt.ylabel('Protein Biomass')
plt.xlabel('Growth Rate')

In [ ]:
condition_df_file = '%s/condition_df_no_gr_norm.csv' % out_loc

if not exists(condition_df_file):

    condition_df = summarize_conditions(df, growth_norm=False)
    condition_df.to_csv(condition_df_file)
else:
    condition_df = pd.read_csv(condition_df_file, index_col=0)
color_df = get_color_for_conditions(condition_df)

condition_df = condition_df.dropna(how='all')

In [ ]:
no_growth = df.loc['biomass_dilution'][df.loc['biomass_dilution'] < .1].sort_values().index
print('Dropping due to low growth', no_growth)
condition_df = condition_df.drop(no_growth)
df = df.drop(no_growth, axis=1)

### (Figure 4) PCA showing how micronutrient use differentiates conditions

In [ ]:
condition_df = condition_df.drop('gthox', axis=1)
condition_df = (condition_df.T / df.loc['protein_biomass_to_biomass']).T.dropna()


In [ ]:

# run PCA
pca_df = condition_df.copy()
expression_values = preprocessing.StandardScaler().fit_transform(pca_df)
pca = PCA(n_components=3)
a = pca.fit(expression_values)
X_r = a.transform(expression_values)


fig = plt.figure(figsize=(10, 10))
gs = fig.add_gridspec(3, 3)
ax = fig.add_subplot(gs[:-1, :-1])

source_to_color = dict(zip(['C_anaerobic', 'C', 'S_anaerobic', 'S', 'P_anaerobic', 'P', 'N_anaerobic', 'N'], 
                          palettable.colorbrewer.qualitative.Paired_8.hex_colors))


# plot and format points on PCA plot
for z in range(len(X_r[:, 0])):
    index = pca_df.index[z]
    source = index.split('_')[0] if 'anaerobic' not in index else '_'.join(index.split('_')[:2]) 
    if 'anaerobic' in source:
        edgecolor = source_to_color[source.split('_')[0]]
        color = 'white'
        label=None
    else:
        edgecolor = None
        label=source_to_name[source]
        color  = source_to_color[source.split('_')[0]]
    ax.scatter(X_r[z, 0], X_r[z, 1], c=color, edgecolors=edgecolor, s=150, label=label, linewidths=1.5, alpha=.7)
    
    
ax.set_xlabel('PC 1 (%.2f)' % a.explained_variance_ratio_[0])
ax.set_ylabel('PC 2 (%.2f)' % a.explained_variance_ratio_[1])
ax.set_facecolor('w')
handles, labels = ax.get_legend_handles_labels()
by_label = OrderedDict(zip(labels[:-1], handles[:-1]))
ax.legend(by_label.values(), by_label.keys(), loc='upper left', facecolor='w', ncol=1, title='Nutrient Sources',
         title_fontsize=13)

# plot PCA weightings
all_weights=pd.DataFrame()
for component in [0, 1]:
    if component == 0:
        ax = fig.add_subplot(gs[-1, :-1])
        bar_kind = 'bar'
    else:
        ax = fig.add_subplot(gs[:-1, -1])
        bar_kind = 'barh'
    ax.set_facecolor('w')
    weight_dfs = pd.DataFrame(a.components_[component], index=pca_df.columns).T
    all_weights = all_weights.join(weight_dfs.T, how='outer', rsuffix='new')
    # from https://www.pnas.org/content/110/6/2135
    # inspired by https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4906156/
    prebiotic_aas = ['ala__L', 'asp__L', 'glu__L', 'gly', 'ile__L', 
                     'leu__L', 'pro__L', 'ser__L', 'thr__L', 'val__L']
    prebiotic_aas = []
    peroyl_scavanging = ['his__L', 'cys__L', 'met__L', 'tyr__L', 'trp__L']
    weight_dfs.sort_values(0, axis=1).loc[0].plot(kind=bar_kind, ax=ax)
    for i, p in enumerate(ax.patches):
        if weight_dfs.sort_values(0, axis=1).columns[i] in prebiotic_aas:
            p.set_color('g')
            p.set_label('Prebiotic AA')
        elif weight_dfs.sort_values(0, axis=1).columns[i] in peroyl_scavanging:
            p.set_color('r')
            p.set_label('Peroxyl scavanging AA')
        elif '__L' not in weight_dfs.sort_values(0, axis=1).columns[i]:
            p.set_color('k')
            p.set_label('Cofactor')
        else:
            p.set_label('Non-prebiotic AA')
    ax.set_title('PC %i weighting' % (component+1))

    if component == 0:
        ax.set_xticklabels(['']* len(weight_dfs.sort_values(0, axis=1).columns))
            # remove duplicate legend entries
        handles, labels = ax.get_legend_handles_labels()
        by_label = OrderedDict(zip(labels[:-1], handles[:-1]))
        ax.legend(by_label.values(), by_label.keys(), ncol=1, facecolor='w')
    else:
        ax.set_yticklabels(['']* len(weight_dfs.sort_values(0, axis=1).columns))


fig.tight_layout()
fig.savefig('%s/figure_4.svg' % out_loc)
all_weights.columns = ['PC 1', 'PC 2']

all_weights.sort_values('PC 1', ascending=False).applymap(lambda x: '%.3f' % x).to_csv(
    '%s/figure_4_pc_weightings.csv' % out_loc)

In [ ]:
all_weights.sort_values('PC 1', ascending=True).head(5)

In [ ]:
from scipy.stats import pearsonr
print(spearmanr(X_r[:, 1], df.loc['biomass_dilution']))
plt.scatter(df.loc['biomass_dilution'], X_r[:, 1])

In [ ]:
temp = pd.DataFrame()
for i in df[df!=0].dropna().index:
    if to_drop(i):
        continue
    rho, p = spearmanr(X_r[:, 1], df.loc[i])
    temp.loc[i, 'rho'] = rho
    temp.loc[i, 'p'] = p

In [ ]:
# plot histograms of specificy metabolites
hist_df = condition_df.copy()
to_plot = list(reversed(['gly', 'nad', 'his__L', 'btn']))
fig, axes = plt.subplots(1, 4, figsize=(10, 1), sharey=True)
for i, col in enumerate(to_plot):
    if col not in to_plot:
        continue
    ax=axes[i]
    ax.set_facecolor('w')
    if col != 'nad':
        ax.set_title(ijo.metabolites.get_by_id(col+'_c').name.replace('amide', 'amide \n'))
    else:
        ax.set_title('NAD')
    ana = [i for i in hist_df.index if 'anaerobic' in i]
    aer = [i for i in hist_df.index if 'anaerobic' not in i]
    ax.hist([hist_df.loc[aer, col], hist_df.loc[ana, col]], bins=20)
    ax.set_xticklabels(['' for i in ax.get_xticks()], 
                  fontdict={'horizontalalignment': 'right', 'rotation':45})
ax.legend(['aerobic', 'anaerobic'], bbox_to_anchor=(-1.25, -.5), loc='upper center', fontsize=15, ncol=2,
         facecolor='w')

fig.savefig('%s/hist_for_pca_figure_4.svg' % out_loc)

## (Figure 3) Standard deviation analysis of nutrient sources

In [ ]:
fig  = plt.figure(figsize=(18,7))
gs = fig.add_gridspec(2, 5)
ax = fig.add_subplot(gs[0,:-1])
#condition_df = condition_df.drop(['C_ethso3_e', 'C_pro__L_e', 'C_for_e', 'N_pro__L_e', 'C_butso3_e'])
plot_df = condition_df.copy()

c2, c1 = palettable.wesanderson.Zissou_5.hex_colors[0:2]
ax = plot_df.max().sort_values().plot(kind='bar', logy=True,
                                      label='max-min', ax=ax, color=c1)

ax = plot_df.min().sort_values().plot(kind='bar', logy=True, ax=ax,
                                      color='w')
ax.set_facecolor('w')
ax.set_ylabel('Normalized biosynthesis \n demand' + r' ($mmol \cdot g_{protein}^{-1}$)', fontsize=18)
ax.set_title('Range of biosynthesis demands', fontsize=18)
ax.set_xticklabels(ax.get_xticklabels(), 
                  fontdict={'fontsize': 18})

aer = [i for i in condition_df.index if 'anaerob' not in i]
anaer = [i for i in condition_df.index if 'anaerob' in i]
all_values ={}
for kind, lis in zip(['aerobic', 'anaerobic'], [aer, anaer]):
    q = condition_df.loc[aer]/ condition_df.loc[aer].T.max(axis=1)
    sources = set([i.split('_')[0] for i in q.index])
    
    sorted_all = q.T.std(axis=1).sort_values().index

    for s in sources:
        source_rxns = [i for i in q.index if i.startswith(s)]
        key = source_to_name[s] if kind == 'aerobic' else source_to_name[s]+' anearobic'
        all_values[key] =dict(zip(sorted_all, list(q.loc[source_rxns, sorted_all].T.std(axis=1).values)))

ax = fig.add_subplot(gs[1,:-1])
all_val_df = pd.DataFrame(all_values)
all_val_df['sum'] = all_val_df.sum(axis=1)
all_val_df = all_val_df.sort_values('sum')
all_val_df[sorted(all_val_df.columns)].drop(['sum'], axis=1).plot.bar(
    stacked=True, colors=palettable.tableau.Tableau_20.hex_colors, ax=ax)
ax.set_facecolor('w')
ax.legend(facecolor='w', ncol=2, fontsize=13)#, title='Nutrient Sources', title_fontsize=12)
ax.set_ylabel('Standard deviation', fontsize=18)
ax.set_title('Standard deviation by nutrient source and aerobicity', fontsize=18)
ax.set_xticklabels(ax.get_xticklabels(), 
                  fontdict={'fontsize': 18})

fig.subplots_adjust(hspace=1)
fig.text(.03, .95, 'A', size=35)
fig.text(.03, .45, 'B', size=35)


for i in plot_df.index:
    plot_df.loc[i, 'aerobicity'] = 'anaerobic' if 'anaerobic' in i else 'aerobic'
plot_df = plot_df.drop('atp', axis=1, errors='ignore')
core_biomass = pd.DataFrame(plot_df.drop('aerobicity', axis=1).min())
core_biomass = core_biomass[core_biomass>0].dropna().sort_values(0, ascending=False)
core_biomass.index.name = 'Metabolite'
core_biomass.columns = ['Demand']
core_biomass = core_biomass.applymap(lambda x: '%.3f' % x if x>1e-3 else '{:.1e}'.format(float(x)))
core_biomass = core_biomass.reset_index()
core_out = pd.DataFrame(index=list(range(20)), columns=list(range(4)))
core_out.iloc[:20, 0] = core_biomass.iloc[:20, 0]
core_out.iloc[:20, 1] = core_biomass.iloc[:20, 1]
core_out.iloc[:10, 2] = core_biomass.iloc[20:, 0].values
core_out.iloc[:10, 3] = core_biomass.iloc[20:, 1].values
core_out.columns = ['AA', 'Demand', 'Cofactor', 'Demand']

skip_c = True
if not skip_c:
    ax = fig.add_subplot(gs[:, -1])
    ax.axis('off')
    ax.axis('tight')
    t =ax.table(cellText=core_out.fillna('').values, colLabels=core_out.columns, loc='lower center')
    t.auto_set_font_size(False)
    t.set_fontsize(13)
    t.scale(1.3,1.4)
    fig.text(.72, .95, 'C', size=35)
    fig.text(.835, .85, 'Core biomass function', size=20, ha='center')

fig.savefig('%s/figure_3.png' % out_loc)
fig.savefig('%s/Fig3.svg' % out_loc)

## Cluster aerobic growth conditions (Figure 5)

In [ ]:
p_cutoff = 1e-5
small_cluster_cutoff= 3
to_skip = ['bmocogdp', 'adocbl', '2dmmq8', 'mqn8', 'sheme']
filt_df = condition_df.loc[[i for i in condition_df.index if 'anaerobic' not in i]]
filt_df = filt_df[[i for i in filt_df if i != 'atp' and i not in to_skip]]
filt_df = filt_df.astype(float) 

z_df = (filt_df - filt_df.mean()) / filt_df.std()
outliers = pd.DataFrame()
for i in z_df.columns:
    if '__L' in i:
        continue
    up = z_df[i][z_df[i] > 3]
    down = z_df[i][z_df[i] < -3]
    #print(i, up, down)
    for cond, val in up.items():
        outliers.loc[i, cond] = val
    for cond, val in down.items():
        outliers.loc[i, cond] = val
    

In [ ]:
fc_df =filt_df.applymap(np.log2) - filt_df.applymap(np.log2).mean()
fc_df = fc_df.T.loc[outliers.index, outliers.columns][~outliers.isna()]
fc_df = fc_df.rename(lambda x: '(%s) ' % x[0] + x[2:], axis=1)

z = sns.clustermap(fc_df.fillna(0), cmap='coolwarm', center=0, figsize=(20,5), method='ward')

fig, ax = plt.subplots(figsize=(20, 4))
g = sns.heatmap(z.data2d[z.data2d !=0], cmap='coolwarm', center=0, ax=ax, linecolor='k',
               linewidths=.5)
ax.set_facecolor('w')
g.collections[0].colorbar.set_label('Log$_2$ fold change from average', rotation=270,
                                    labelpad=20,fontsize=20)
ax.set_title('Biomass biosynthetic demand outliers', fontsize=20)
fig.savefig('%s/outliers.svg' % out_loc)

In [ ]:
thf_outlier = outliers.loc['thf'].dropna().index

In [ ]:
pydx5_outlier = outliers.loc['pydx5p'].dropna().index
non_pydx5_outlier = [i for i in condition_df.index if i not in pydx5_outlier] 
for i in pydx5_outlier:
    print(i, ijo.metabolites.get_by_id(i[2:]).name)
print(condition_df.loc[pydx5_outlier, 'pydx5p'].mean() / condition_df.loc[
    non_pydx5_outlier, 'pydx5p'].mean())

In [ ]:
pydx_df = pd.DataFrame()

for i in me.reactions.query('mod_pydx5p'):
    r_name = i.id.split('_FWD')[0].split('_REV')[0]
    if not isinstance(i, cobrame.MetabolicReaction):
            continue
    for j in pydx5_outlier:#condition_df.index:
        
        pydx_df.loc[r_name, j] = df.loc[i.id, j]
pydx_df = pydx_df[pydx_df != 0].dropna(how='all')

pydx_df['sum'] = pydx_df[pydx5_outlier].mean(axis=1, skipna=True)
for i in pydx_df.index:
    pydx_df.loc[i, 'name'] = ijo.reactions.get_by_id(i).name
#(pydx_df[pydx5_outlier].mean(axis=1, skipna=True) / pydx_df[non_pydx5_outlier].mean(axis=1, skipna=True)).sort_values()#

In [ ]:


norm_df = filt_df / filt_df.max()
norm_df = norm_df.drop(outliers.columns)
no_aas = False
if no_aas:
    clus_num=8
    aas = [i[:-2] for i in amino_acids.values()]
    norm_df = norm_df[[i for i in norm_df.columns if i not in aas]]
    norm_df.to_csv("%s/normalized_condition_no_aas_df.csv" % out_loc)
else:
    clus_num = 6
    norm_df.to_csv("%s/normalized_condition_df.csv" % out_loc)


#z_df = pd.DataFrame(z_df, index=filt_df.index, columns=filt_df.columns)
summary=pd.DataFrame()

for n_clus in [clus_num]:
    fit = AgglomerativeClustering(n_clusters=int(n_clus), linkage='ward',
                                  affinity='euclidean').fit(norm_df)

    out_df = pd.DataFrame([norm_df.index, fit.labels_], index=['met', 'cluster']).T

    drop_indexes = []
    p_df = pd.DataFrame()
    new_df = pd.DataFrame()
    sizes = []
    for clust, temp_df in out_df.groupby('cluster'):
        mets = [i for i in temp_df.met]
        
        non_clust_mets = [i for i in out_df.met if i not in mets]

        sizes.append(len(mets))
        new_df.loc[clust, 'mets'] = ', '.join(mets)
        for cofactor in norm_df.columns:

            clust_values = norm_df.loc[mets, cofactor].values
            non_clust_values = norm_df.loc[non_clust_mets, cofactor].values
            #p_df.loc[clust, cofactor] = ks_2samp(clust_values, non_clust_values)[1]
            p_df.loc[clust, cofactor] = ranksums(clust_values, non_clust_values)[1]

            #p_df.loc[clust, cofactor] = ttest_1samp(clust_values, norm_df.loc['C_glc__D_e', cofactor])[1]
            new_df.loc[clust, cofactor] = np.log2(clust_values.mean() / non_clust_values.mean())# -1 #non_clust_values.mean()
    
    binary_p_df = p_df.copy()
    binary_p_df[binary_p_df > p_cutoff] = 0
    binary_p_df[(binary_p_df <= p_cutoff) & ( binary_p_df >0)] = 1
    clust_cols = [i for i in new_df.columns if i!='mets']
    binary_p_df[(new_df[clust_cols]>-.15) & (new_df[clust_cols] <.15)] = 0
            
    summary.loc[n_clus, 'num_sig'] = binary_p_df.sum(axis=1).sum()
    summary.loc[n_clus, 'num_clus'] = len(p_df.index)
    summary.loc[n_clus, 'avg_num_sig'] = binary_p_df.sum(axis=1).mean()
    summary.loc[n_clus, 'avg_size'] = np.array(sizes).mean()
    summary.loc[n_clus, 'num_sig_clus'] = (binary_p_df.sum(axis=1) > 0).sum()
    
clus_to_skip = []
for i in new_df.index:
    if len(new_df.loc[i, 'mets'].split(', ')) <= small_cluster_cutoff:
        clus_to_skip.append(i)
        
print(len(new_df.index))
summary




In [ ]:
# cluster 2 vs 5 differences
t_df = (df / df.loc['protein_biomass_to_biomass'])
q = (t_df[new_df.loc[1, 'mets'].split(', ')].mean(axis=1).map(np.log2) - 
     t_df[new_df.loc[4, 'mets'].split(', ')].mean(axis=1).map(np.log2)).dropna().sort_values()

q = q[q.abs() != np.inf]
q.loc[[i for i in q.index if not to_drop(i)]].sort_values(ascending=False).head(10)

In [ ]:
biomass_compositions = pd.DataFrame()
for cluster_num in new_df.index:
    mets = new_df.loc[cluster_num, 'mets'].split(', ')
    column_name = 'Cluster ' + str(cluster_num + 1)
    biomass_compositions[column_name] = condition_df.drop(to_skip,axis=1).loc[mets].mean()
    biomass_compositions.loc['Conditions in cluster', column_name] =  new_df.loc[cluster_num, 'mets']
    
biomass_compositions.T.to_excel('%s/S2_data.xlsx' % out_loc)

In [ ]:

for i in new_df.index:
    changes = binary_p_df.loc[i][binary_p_df.loc[i] > 0].index
    add_str = ''
    for change in changes:
        add_str += ', %s (%.2f)' % (change, new_df.loc[i, change])
    new_df.loc[i, 'changes'] = add_str.strip(', ')
save_df = new_df[['mets', 'changes']]
save_df.index.name = 'Cluster'
save_df.index = save_df.index.map(lambda x: x+1)
save_df['n_mets'] = save_df.mets.map(lambda x: len(x.split(', ')))
def convert_for_table(x):
    out = ''
    for i in x.split(', '):
        out += '(%s) %s, ' % (i[0], ijo.metabolites.get_by_id(i[2:]).name)
    return out
save_df.mets = save_df.mets.map(convert_for_table)
save_df.to_csv('%s/cluster_df.csv' % out_loc)


In [ ]:
save_df

In [ ]:

n_big_clus = len(new_df.index) - len(clus_to_skip)
fig, axes = plt.subplots( 1,n_big_clus, figsize=(10, 4), sharey=True)
axes = axes.flatten()
count=0
all_changed = []
for q in new_df.index:
    all_changed.extend([i.split(' (')[0] for i in new_df.loc[q, 'changes'].split(', ')] )
all_changed = [i for i in all_changed if i != '']
plot_df = pd.DataFrame(index=set(all_changed))

for i in new_df.index:
    if i in clus_to_skip:
        continue
        
    cluster = i
    clusts = new_df.loc[cluster, 'mets'].split(', ')
    changed = [i.split(' (')[0] for i in new_df.loc[cluster, 'changes'].split(', ')]
 
    log_df = norm_df.copy()
    plot_df.iloc[:, :] = 0
    if changed ==['']:
        print(clusts)
        count+=1
        continue
    
    nonclus_mets = [i for i in log_df.index if i not in clusts]
    plot_df['mean_clus'] = np.log2(log_df.loc[clusts, changed].mean()) - np.log2(log_df.loc[non_clust_mets, changed].mean())
    
    temp_values = log_df.loc[clusts, changed] / log_df.loc[nonclus_mets, changed].mean() #-1
    plot_df['mean_clus'] = temp_values.applymap(np.log2).mean()
    plot_df['std_clus'] = temp_values.applymap(np.log2).std()
    plot_df['mean_nonclus'] = log_df.loc[nonclus_mets, changed].mean()#log_df.loc['C_glc__D_e', changed]
    plot_df['std_nonclus'] = log_df.loc[nonclus_mets, changed].std()
    #plot_df.plot(kind='barh', y=['mean_clus', 'mean_nonclus'], 
    #             xerr=[plot_df['std_clus'], plot_df['std_nonclus']], ax=axes[count])
    plot_df.plot(kind='barh', y=['mean_clus'], xerr=plot_df['std_clus'], ax=axes[count])
    axes[count].yaxis.set_tick_params(labelsize=12)    
    axes[count].legend_ = None
    axes[count].set_title('Cluster %i' % (cluster+1), fontsize=14)
    axes[count].plot([0,0], [-1, len(all_changed)], 'k--')
    #axes[count].set_xlim([0, 1])
    count+=1

#axes[count-1].legend(['In Cluster', 'Not In Cluster'], fontsize=10, ncol=1, bbox_to_anchor=(1,.5), loc='center left',
#                    title='Growth Conditions')
#axes[count-1].s
fig.tight_layout()
fig.text(.43, -.05, 'Log$_2$ fold change from average', ha='center', fontsize=15)
fig.savefig('%s/cluster_bar_graph.svg' % out_loc)

In [ ]:
binary_columns = condition_df[condition_df==0].dropna(axis=1, how='all').columns
binary_heat_df = condition_df[binary_columns].copy()
binary_heat_df[binary_heat_df>0] = 1
binary_heat_df.columns = [met_to_name[i.replace('_c', '')]
                          for i in binary_heat_df.columns]
g = sns.clustermap(binary_heat_df, row_colors=color_df, yticklabels=False,
                  cmap=palettable.cartocolors.diverging.Geyser_3.get_mpl_colormap(),
                  method='ward')


g.ax_col_dendrogram.set_visible(False)
g.ax_row_dendrogram.set_visible(False)
g.cax.clear()
g.cax.axis('off')
g.cax.text(18, -1, 'Cofactor required', fontdict={'fontsize': 18})
g.cax.text(7, -1, 'Cofactor not required', fontdict={'fontsize': 18})
q = plt.setp(g.ax_heatmap.get_xticklabels(), rotation=45, ha='right')
q = plt.setp(g.ax_row_colors.get_xticklabels(), rotation=45, ha='right')
plt.savefig('%s/conditionally_essential_cofactors.png' % out_loc)

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(10, 8))
binary_p_df.sum().sort_values().plot(kind='bar', ax=axes[0])
axes[0].set_title('Number of clusters with signficant differences in metabolite')


binary_p_df.rename(lambda x: 'Cluster ' + str(x+1)).T.sum().sort_values().plot(kind='bar', ax=axes[1])
axes[1].set_title('Number of significant metabolite differences per cluster')

fig.tight_layout()